#### Combine the cricket datasets
#### Filter the data from two countries (country columnn) : India and Australia
#### Find the list of bowlers who have taken at least 25 wickets overall(sum).check if your results are correct by comparing with sites such as CricInfo(.g wickets taken by the bowler who retired before 2020)
#### Take a random sample of 30 such indian bwlers and 30 australian bowlers
#### Find if the bowling average of these sampled Indians and Australians show a Statistical significance differnce (two-Sampled test)
#### Bowling average = runs conceded/wickets taken
#### Extend this to include one more country(eg England). Now find if the bowling average of these sampled Indian, Australian and english bowlers show the a statistically signigicant difference(ANOVA)

In [1]:
# Importing necessary packages and methods
import pandas as pd


In [105]:
# Reading the data from csv files
Men_ODI_20 = pd.read_csv('Men ODI Player Innings Stats - 20th Century.csv')
Men_ODI_21 = pd.read_csv('Men ODI Player Innings Stats - 21st Century.csv')

# Selecting only the relevant columns (player name, runs conceded, wickets taken)
Men_ODI_20_selected = Men_ODI_20[['Innings Player', 'Country', 'Innings Runs Conceded', 'Innings Wickets Taken']]
Men_ODI_21_selected = Men_ODI_21[['Innings Player', 'Country', 'Innings Runs Conceded', 'Innings Wickets Taken']]

# Concating the two dataframes
df_selected = pd.concat([Men_ODI_20_selected, Men_ODI_21_selected])

# Filtering the countries column (india and australia)
df = df_selected[(df_selected['Country'] == 'India') | (df_selected['Country'] == 'Australia')]

# Printing few rows
df.head(10)

,Innings Player,Country,Innings Runs Conceded,Innings Wickets Taken
3542,AC Gilchrist,Australia,NaN,NaN
3543,DM Jones,Australia,NaN,NaN
3544,RT Ponting,Australia,NaN,NaN
3545,GS Chappell,Australia,NaN,NaN
3546,AC Gilchrist,Australia,NaN,NaN
3547,ME Waugh,Australia,NaN,NaN
3548,ME Waugh,Australia,NaN,NaN
3549,AC Gilchrist,Australia,NaN,NaN
3550,AR Border,Australia,NaN,NaN
3551,GR Marsh,Australia,NaN,NaN


#### Doing EDA

In [106]:
df.shape # we have 87158 records

(87158, 4)

In [107]:
len(df['Innings Player'].unique()) # there are 459 unique players

459

#### Hadling the null values

In [108]:
df['Innings Runs Conceded'].isnull().sum() # there are 43579 null values in the 'Runs Conceded' Column

np.int64(43579)

In [109]:
df['Innings Wickets Taken'].isnull().sum() # theres are 43579 null values in the 'Wickets Taken' Column

np.int64(43579)

#### Filling the na values with zero

In [110]:
df = df.fillna(0)

In [111]:
df['Innings Wickets Taken'].unique()

array([0, '6', '5', '4', '3', '2', '1', '0', '-', '7'], dtype=object)

#### Handling '-' in the data

In [112]:
df['Innings Wickets Taken'].replace('-', 0, inplace=True)

C:\Users\Kartik\AppData\Local\Temp\ipykernel_23160\3900186089.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Innings Wickets Taken'].replace('-', 0, inplace=True)


In [113]:
df['Innings Runs Conceded'].replace('-', 0, inplace=True)

C:\Users\Kartik\AppData\Local\Temp\ipykernel_23160\910964966.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Innings Runs Conceded'].replace('-', 0, inplace=True)


#### Type casting the Runs and Wickets Columns to int

In [114]:
df['Innings Runs Conceded'] = df['Innings Runs Conceded'].astype(int)
df['Innings Wickets Taken'] = df['Innings Wickets Taken'].astype(int)

In [117]:
df

,Innings Player,Country,Innings Runs Conceded,Innings Wickets Taken
3542,AC Gilchrist,Australia,0,0
3543,DM Jones,Australia,0,0
3544,RT Ponting,Australia,0,0
3545,GS Chappell,Australia,0,0
3546,AC Gilchrist,Australia,0,0
...,...,...,...,...
277077,MK Pandey,India,0,0
277078,MA Agarwal,India,0,0
277079,KL Rahul,India,0,0
277080,SS Iyer,India,0,0


In [124]:
#### Grouping by the player name
grouped_df = df.groupby('Innings Player').agg({
    'Country': 'first',  # Retain the first occurrence of 'Country'
    'Innings Runs Conceded': 'sum',
    'Innings Wickets Taken': 'sum'
})

#### Filtering the bowlers who have wickets more than 25

In [126]:
top_bowlers = grouped_df[grouped_df['Innings Wickets Taken'] > 25]
top_bowlers

,Country,Innings Runs Conceded,Innings Wickets Taken
Innings Player,,,
A Kumble,India,15246,454
A Mishra,India,4533,192
A Nehra,India,14697,465
A Symonds,Australia,13659,353
A Zampa,Australia,7862,218
...,...,...,...
XJ Doherty,Australia,6672,165
YK Pathan,India,4095,99
YS Chahal,India,7053,273


In [132]:
# Sampling (30 players from each country)
indian_bowlers = top_bowlers[top_bowlers['Country'] == 'India']
australian_bowlers = top_bowlers[top_bowlers['Country'] == 'Australia']

### Take a random sample of 30 such indian bowlers and 30 australian bowlers

In [134]:
# Group by 'Country' and take a sample of 30 players from each country
ind_sample = indian_bowlers.sample(n=30, random_state=42)
aus_sample = australian_bowlers.sample(n=30, random_state=42)

In [137]:
aus_sample.shape

(30, 3)

In [142]:
pip install scipy

     ---------------------------------------- 46.2/46.2 MB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [146]:
ind_sample['bowling_avg'] = ind_sample['Innings Runs Conceded'] / ind_sample['Innings Wickets Taken']
aus_sample['bowling_avg'] = aus_sample['Innings Runs Conceded'] / aus_sample['Innings Wickets Taken']

### Find if the bowling average of these sampled Indians and Australians show a Statistical significance differnce (two-Sampled test)

In [151]:
import scipy.stats as stats

t_stat, pvalue = stats.ttest_ind(ind_sample['bowling_avg'], aus_sample['bowling_avg'])

In [152]:
print('t_stat', t_stat)
print('pvalue', pvalue)

t_stat 3.218281321639844
pvalue 0.0021125639989687914


In [157]:
alpha = 0.05

if pvalue < alpha:
    print('Reject the null hypothesis. There is significant difference between the indian bowlers\' bowling avg and the australian bowlers\' bowling avg')
else:
    print('Fail to reject the null hypothesis. There is no significant difference between the indian bowlers\' bowling avg and the australian bowlers\' bowling avg')

Reject the null hypothesis. There is significant difference between the indian bowlers' bowling avg and the australian bowlers' bowling avg


In [159]:
ind_sample['bowling_avg'].mean()

np.float64(35.06864445698241)

In [160]:
aus_sample['bowling_avg'].mean()

np.float64(28.86136389993946)

### Extend this to include one more country(eg England). Now find if the bowling average of these sampled Indian, Australian and english bowlers show the a statistically signigicant difference(ANOVA)

In [178]:
req_countries = ['India', 'Australia', 'England']
df2 = df_selected[df_selected['Country'].isin(req_countries)]

In [190]:

# Cleansing 

df2['Innings Runs Conceded'].replace('-', 0, inplace=True)
df2['Innings Wickets Taken'].replace('-', 0, inplace=True)

# filling na
df2 = df2.fillna(0)

df2['Innings Runs Conceded'] = df2['Innings Runs Conceded'].astype(int) # typecasting
df2['Innings Wickets Taken'] = df2['Innings Wickets Taken'].astype(int)


# grouping by player name
grouped_df2 = df2.groupby('Innings Player').agg({
    'Country': 'first',  # Retain the first occurrence of 'Country'
    'Innings Runs Conceded': 'sum',
    'Innings Wickets Taken': 'sum'
})

# filtering the top bowlers (wickets > 25)
top_bowlers = grouped_df2[grouped_df2['Innings Wickets Taken'] > 25]
top_bowlers

# sampling for english bowlers
english_bowlers = top_bowlers[top_bowlers['Country'] == 'England']
eng_sample = english_bowlers.sample(n=30, random_state=42)

# calculating bowling_avg
eng_sample['bowling_avg'] = eng_sample['Innings Runs Conceded'] / eng_sample['Innings Wickets Taken']

C:\Users\Kartik\AppData\Local\Temp\ipykernel_23160\1110631324.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2['Innings Runs Conceded'].replace('-', 0, inplace=True)
C:\Users\Kartik\AppData\Local\Temp\ipykernel_23160\1110631324.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [191]:
eng_sample

,Country,Innings Runs Conceded,Innings Wickets Taken,bowling_avg
Innings Player,,,,
AF Giles,England,5577,149,37.429530
JE Root,England,4335,72,60.208333
BA Stokes,England,8760,210,41.714286
A Flintoff,England,11408,490,23.281633
MA Ealham,England,2503,69,36.275362
MH Yardy,England,3225,63,51.190476
RL Johnson,England,717,33,21.727273
PAJ DeFreitas,England,3775,115,32.826087
MP Vaughan,England,1947,48,40.562500


In [192]:
# Now we have ind_sample, aus_sample and eng_sample 
# so we are ready for the ANOVA test

f_stat, pvalue = stats.f_oneway(ind_sample['bowling_avg'], aus_sample['bowling_avg'], eng_sample['bowling_avg'])
print('f_stat', f_stat)
print('pvalue', pvalue)

f_stat 6.39403509593666
pvalue 0.0025654724257179904


In [194]:
alpha = 0.05

if pvalue < alpha:
    print('Reject the null hypothesis. There is significant difference between the indian bowlers\' bowling avg, the australian bowlers\' bowling avg and the english bowlers\' bowling avg')
else:
    print('Fail to reject the null hypothesis. There is no significant difference between the indian bowlers\' bowling avg, the australian bowlers\' bowling avg and the english bowlers\' bowling avg')

Reject the null hypothesis. There is significant difference between the indian bowlers' bowling avg, the australian bowlers' bowling avg and the english bowlers' bowling avg


In [196]:
ind_sample['bowling_avg'].mean()

np.float64(35.06864445698241)

In [197]:
aus_sample['bowling_avg'].mean()

np.float64(28.86136389993946)

In [198]:
eng_sample['bowling_avg'].mean()

np.float64(35.53596530927358)